# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob, types
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch, Arc
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','MotorPreserved.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

plotting group double errorbars for speed 

In [ ]:
def plot_event_1on1(root, ax, Profiles, colorCode, badAnimals=None, TaskParamToPlot='% good trials',
                   x_pos=None, nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(3,6),
                    seed=1, animal_plot=True):
    if badAnimals is None:
        badAnimals=[]
    
    if x_pos is None:
        diff=.35
        x_c=ax.get_xlim()[1]
        x_pos=(x_c-diff,x_c+diff,x_c+3*diff)
    diff=x_pos[1]-x_pos[0]
    
    #getting the data
    animalList,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    
    data=np.empty((len(animalList),3))
    for i,animal in enumerate(animalList):
        preSession,postSession=sessionDict[animal][0], sessionDict[animal][1]
        out=animal_learning_stats(root, animal, PerfParam=TaskParamToPlot,
                                  goodSessions=[*preSession,*postSession],redo=False)
        
        data[i,0]=np.nanmedian(out[:len(preSession)][nPre])
        data[i,1]=np.nanmedian(out[len(preSession):][nPost])
        data[i,2]=np.nanmedian(out[len(preSession):][nFin])
        
            
    
    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='o', zorder=5, ms=2, elinewidth=1, color='k')
    
    #plotting individual animals
    if animal_plot:
        np.random.seed(seed=seed)
        _coeff=5
        for i in range(data.shape[0]):
            if np.any(np.isnan(data[i,:])):
                continue
            jitPre =np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            jitFin =np.random.uniform(low=x_pos[2]-diff/_coeff, high=x_pos[2]+diff/_coeff, size=1)
            
            _,tag=lesion_type(root,animalList[i])
            c=colorCode[tag] if tag in colorCode else 'gray'
            ax.scatter([jitPre,jitPost,jitFin],data[i,:], s=1, c=c, marker='o', edgecolors='none', alpha=.8)
            ax.plot([jitPre,jitPost,jitFin],data[i,:], c=c, lw=.2, alpha=.4)
#             ax.plot([jitPre,jitFin],data[i,[0,-1]], c=c, lw=.2, alpha=.4)

    ax.set_xlim([x_pos[0]-diff/2,x_pos[-1]+diff/2])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Before','After','Final'])
    ax.xaxis.set_tick_params(rotation=0)
    ax.spines['bottom'].set_bounds(x_pos[0],x_pos[-1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylabel(TaskParamToPlot)

    return data,animalList

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse','Reverse-AfterBreak']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse-Late_DLS']
             }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot='median entrance time (sec)'
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    ax.set_xticks([])
    a=plot_event_1on1(root, ax, Profiles, colorCode={'':'xkcd:aqua','DLS':'xkcd:aqua'}, badAnimals=None, TaskParamToPlot=TaskParamToPlot,
                       x_pos=None, nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(7,10),
                        seed=1, animal_plot=True)

---

speed vs max pos DEFINITION

In [ ]:
def plot_speed_pos_def(ax):
    step=.01
    x0=np.arange(0,7,step)
    y0=10*x0+10

    x1=np.arange(7, 8.16,step)
    y1=-60*x1+500

    x=np.append(x0,x1)
    y=np.append(y0,y1)
    y=smooth(y*1.1,sigma=50,mode='constant')


    _x0=np.arange(0,2.5,step)
    _y0=10*_x0+15

    _x1=np.arange(2.5, 9,step)
    _y1=-3.85*_x1+49.6

    _x=np.append(_x0,_x1)
    _y=np.append(_y0,_y1)

    _y=smooth(_y*1.1,sigma=50,mode='nearest')    
    
    
    #plot the traj
    ax.plot(x, y, lw=2, c='xkcd:navy', )
    ax.plot(_x, _y, lw=2, c='xkcd:teal', )
    #plot the horizontal lines
    ax.axhspan(ymin=70, ymax=90, ec=None, fc='gray', alpha=.2)
    ax.axhspan(ymin=40, ymax=60, ec=None, fc='gray', alpha=.2)
    #plot the red piece
    x7=np.where(x>=7.3)[0][0]
    _x4=np.where(_x>=4.5)[0][0]
    LW=ax.spines['bottom'].get_lw()
    ax.plot(x[x7:x7+int(.3/step)],y[x7:x7+int(.3/step)],'r',lw=LW,zorder=5)
    ax.plot(_x[_x4:_x4+int(1.5/step)],_y[_x4:_x4+int(1.5/step)],'r',lw=LW,zorder=5)
    #plot the x-axis at the red piece
    ax.arrow(x[x7],y[x7],1.3,0,lw=LW,head_width=2,head_length=.2, ec='r', fc='r',zorder=5)
    ax.arrow(_x[_x4],_y[_x4],1.3,0,lw=LW,head_width=2,head_length=.2, ec='r', fc='r',zorder=5)
    # plot the curve
    ax.add_patch(Arc(xy=(x[x7],y[x7]), width=1, height=10, angle=0, theta1=271, theta2=360,
                     lw=LW, color='r', zorder=7))
    ax.add_patch(Arc(xy=(_x[_x4],_y[_x4]), width=1.5, height=15, angle=0, theta1=282, theta2=360,
                     lw=LW, color='r', zorder=7))
    #writing 
    ax.text(0,50,'Mid',rotation=0, c='k', fontsize='xx-small',ha='left',va='center')
    ax.text(0,80,'Back',rotation=0, c='k', fontsize='xx-small',ha='left',va='center')

    
    ax.set_xticks([0,1,2,3,4,5,6,7,8,9])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'',20,'',40,'',60,'',80,''])
    ax.spines['bottom'].set_bounds(0,9)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.set_xlim([-.2,9])
    ax.set_ylim([-2,90])
    ax.set_ylabel('Position (cm)')
    ax.set_xlabel('Trial time (s)')

In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(1.5,1.5),dpi=600)
    ax=fig.add_subplot(111)
    plot_speed_pos_def(ax)

---

plot speed vs max pos trial-by-trial

In [ ]:
_MinX_=30
def maxPos_speed_correlation(data):
    _,maxTrials,maxPos=sequentialTrials(data, xFront=_MinX_)._compute_max_pos()
    spd=forwardRunningSpeed(data,minXBack=_MinX_+10,minXFront=_MinX_).compute()
    
    MaxTrialSpd={trial:spd[trial] for trial in maxTrials if trial in spd.keys()}
    MaxTrialPos=np.array([maxPos[i] for i,trial in enumerate(maxTrials) if trial in MaxTrialSpd.keys()])
    MaxTrialSpd=np.array(list(MaxTrialSpd.values()))
        
    return MaxTrialPos,MaxTrialSpd

def plot_animal_maxPos_speed_relation(root, ax, profile, colorCode, Slice=slice(-5,None),animal_plot=False):
    FrontBin=np.array([_MinX_+10,_MinX_+30])
    BackBin =np.array([70,90])
    MinNbTrials=10
    x_pos=[0,1]
    diff=x_pos[1]-x_pos[0]
    
    animalList=batch_get_animal_list(root, profile)
    
    pos={}
    spd={}
    for animal in animalList:
        data=data_fetch(root, animal, profile, PerfParam=[maxPos_speed_correlation], NbSession=Slice)['maxPos_speed_correlation']
        
        pos[animal]=np.array([i for sessionData in data for i in sessionData[0]])
        spd[animal]=np.array([i for sessionData in data for i in sessionData[1]])        
    
    data=np.ones((len(spd),2))*np.nan
    for i,(animal,p) in enumerate(pos.items()):
        
        frontTrials=np.where(np.logical_and(p>=FrontBin[0], p<=FrontBin[1]))[0]
        if len(frontTrials) > MinNbTrials:
            data[i,0]=np.nanmean(spd[animal][frontTrials])

        backTrials=np.where(np.logical_and(p>=BackBin[0], p<=BackBin[1]))[0]
        if len(backTrials) > MinNbTrials:
            data[i,1]=np.nanmean(spd[animal][backTrials])


    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='o', zorder=5, ms=2, elinewidth=1, color='k')
    
    #plotting individual animals
    if animal_plot:
        np.random.seed(seed=3)
        _coeff=5
        for i in range(data.shape[0]):
            if np.any(np.isnan(data[i,:])):
                continue
            jitPre =np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            
            _,tag=lesion_type(root,animalList[i])
            c=colorCode[tag] if tag in colorCode else 'gray'
            ax.scatter([jitPre,jitPost],data[i,:], s=1, c=c, marker='o', edgecolors='none', alpha=.8)
            ax.plot([jitPre,jitPost],data[i,:], c=c, lw=.2, alpha=.4)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([x_pos[0]-diff/4,x_pos[-1]+diff/4])
    ax.set_xticks(x_pos)
    ax.spines['bottom'].set_bounds(*x_pos)
    ax.set_xticklabels(['Mid','Back'])
#     ax.set_xlabel('Max. Pos.')
    ax.xaxis.set_tick_params(rotation=0)

            
    return pos,spd

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    a=plot_animal_maxPos_speed_relation(root, ax, profile1, color, Slice=slice(-3,None),animal_plot=True)

---

plot consecutive trials

In [ ]:
def plot_consecutive_trajectories(root, session, trials, ax):
    delta=1 #last delta second of position is not recorded
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    data.position_correction()
    try:
        detect_trial_end(data, trials)
    except:
        pass
    fps=data.cameraSamplingRate

    maxT=0
    for trial in trials:
        time=data.rawTime[trial]+maxT
        maxT=time[-1]+delta
        #plotting position
        ax.plot(time,data.position[trial],'k');
        #plotting entrance time
        ET=data.entranceTime[trial]+data.cameraToTreadmillDelay+time[0]
        if data.entranceTime[trial]!=data.maxTrialDuration[trial]:
            ax.plot(ET,data.position[trial][time[time<ET].argmax()+1],'rx')
        #plotting the goal time
        GT=data.goalTime[trial]+data.cameraToTreadmillDelay+time[0]
        ax.plot(GT,data.position[trial][time[time<GT].argmax()], marker='o',markeredgecolor='r', markerfacecolor='None')
        #plotting the highlight for trials
        x=(time[0], time[0]+data.timeEndTrial[trial]+data.cameraToTreadmillDelay)
        c='xkcd:green' if trial in data.goodTrials else 'salmon' if data.entranceTime[trial]<data.goalTime[trial] else 'cyan'
        ax.fill_betweenx(y=(Y1,Y2),x1=x[0],x2=x[1], facecolor=c, edgecolor=None, alpha=.2)
    
    ax.set_xlim([-1,time[-1]+1])
    ax.set_ylim([-2,92])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'','','','','','','','',90])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (cm)')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.spines['bottom'].set_bounds(0,time[-1])
    
    return ax
    
    
def add_legend_for_consecutive_trajectories(ax):
    o_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='r', marker='o',
                                       markerfacecolor='None', markersize=6, label='Goal time')

    x_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='r', marker='x',
                                       markerfacecolor='None', markersize=6, label='ET')


    g_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='xkcd:green', label='Correct trial', alpha=1)
    r_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='salmon', label='Error trial', alpha=1)
#     b_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='cyan', label='Omission trial', alpha=1)
    w_patch = matplotlib.patches.Patch(edgecolor='k', facecolor='white', label='Intertrial', alpha=1)



    leg=ax.legend(handles=[g_patch,r_patch,w_patch,o_marker,x_marker],loc=(.85,.1),#mode='expand',
                  ncol=1, fontsize='xx-small', framealpha=.6,frameon=True)
    return leg

In [ ]:
if "__file__" not in dir():
    session='Rat366_2019_10_02_10_50'
    i=90
    trials=range(i,i+9)

    plt.close('all')
    ax=plt.figure(figsize=(20,3)).add_subplot(111);

    plot_consecutive_trajectories(root, session, trials, ax)
    add_legend_for_consecutive_trajectories(ax)

---

plot single session trajectories

In [ ]:
def plot_session_median_trajectory(data,ax):
    posDict=data.position
    maxL=np.nanmax(list(data.stopFrame.values()))
    maxL=int(maxL)
    position=np.ones((maxL,len(posDict.keys())))*np.nan
    time=np.arange(-data.cameraToTreadmillDelay,
                   (maxL-data.cameraSamplingRate)/data.cameraSamplingRate,
                   1/data.cameraSamplingRate)
    
    
    for i,trial in enumerate(posDict):
        pos=posDict[trial][:data.stopFrame[trial]]
        position[:len(pos),i]=pos
    
    #keeping data where 70% of points exist
    nanSum=np.sum(np.isnan(position),axis=1)
    try:
        maxTraj=np.where(nanSum>.3*position.shape[1])[0][0]
    except IndexError:
        maxTraj=position.shape[1]
    
    
    ax.plot(time[:maxTraj], np.nanmedian(position,axis=1)[:maxTraj], color='navy', lw=2)    

def plot_trajectories(data,ax):
    posDict=data.position
    time=data.timeTreadmill #align on camera
    Colors=[]
    for trial in posDict:
        color="xkcd:green"
        if trial not in data.goodTrials:
            color="salmon"
        Colors.append(color)
        ax.plot(time[trial][:data.stopFrame[trial]], posDict[trial][:data.stopFrame[trial]],
               color=color, lw=.5, )
            
    ax.fill_betweenx(y=(0,90),x1=0,x2=7, facecolor='gray', edgecolor=None, alpha=.4)
    
    return np.array(Colors)



def plot_trajectories_and_distributions(root, ax, session, showText=True):
    data=Data(root,session[:6],session,redoPreprocess=False)
    
    color=plot_trajectories(data,ax=ax)
    
    position=get_positions_array_beginning(data,onlyGood=False,raw=False)
    position=position.T
    
    plot_session_median_trajectory(data,ax)
    
    props={'color':'k', 'linewidth':1}
    ax.boxplot(x=data.entranceTime,whis=[5,95],vert=False,
               positions=[5], widths=5,
               showcaps=False, showfliers=False,
               medianprops=props, boxprops=props, whiskerprops=props, zorder=5
              )
    
    if showText:
        ax.set_xlim([-1.2,13.2])
        ax.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13])
        ax.set_xticklabels([0,'','','','','','',7,'','','','','',13])
        ax.set_ylim([-2,92])
        ax.set_yticks([0,10,20,30,40,50,60,70,80,90])
        ax.set_yticklabels([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_bounds(0,13)
        ax.set_xlabel('Trial time (s)',labelpad=0)
#         ax.set_ylabel('Position (cm)',labelpad=0)
        ax.spines['left'].set_bounds(0,90)

    else:
        ax.set_xlim([-1,15.2])
        ax.set_ylim([-2,92])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.tick_params(bottom=False, top=False, left=False, right=False,
                      labelbottom=False, labeltop=False, labelleft=False, labelright=False)
    


In [ ]:
if "__file__" not in dir():
    session='Rat366_2019_10_02_10_50'

    plt.close('all')
    ax=plt.figure(figsize=(3,3)).add_subplot(111);

    plot_trajectories_and_distributions(root, ax, session, showText=True)

---

AntiSequential trials: Starting from Back for the Reverse Treadmill

In [ ]:
def _anti_sequential_trials(data, xFront=60):
    data.position_correction()
    detect_trial_end(data,trials=data.position.keys())
    
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)

    seqTrial=[]
    for trial,pos in data.position.items():
        #starting from the front
        if pos[trialStart]<xFront and pos[0]<xFront:
            continue

        #not completing the trial
        if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
            continue

        #The trial IS indeed sequential
        seqTrial.append(trial)

    return len(seqTrial)/data.nTrial

In [ ]:
if "__file__" not in dir():
    session='Rat366_2019_10_02_10_50'
    data=Data(root,session[:6],session,redoPreprocess=False)
    
    print(_anti_sequential_trials(data))

---

Free Exploration on immobile treadmill

In [ ]:
def _find_persistent_movement_epochs(pos: np.array, TrdSpeed: int, fs: int):
    """
    this function finds epochs in the position of a single trial, pos, where the animal has moved
    for a duration of at least, TH*cameraSamplingRate
    pos: position as a np.array
    """
    SpeedTolerance = 5 if TrdSpeed==0 else 10
    DurationTolerance= 0.4*fs
    
    pos_=pos#smooth(pos,.3*fs,mode="nearest")
    velocity=np.append(TrdSpeed-(np.diff(pos_)*fs),TrdSpeed)
    
    zeroVel=np.logical_and(velocity<SpeedTolerance, velocity>-SpeedTolerance)
    
    movement= np.logical_not(zeroVel)
    movement=movement*1

    dpcross=np.absolute( np.append(movement,0) - np.append(0,movement) )
    #any 1 in dpcross mean a zero-crossing in diff(pos)
    crossindex=np.where(dpcross)[0]
    crossindex=np.reshape(crossindex,(-1,2))
    movementepoch=crossindex[crossindex[:,1]-crossindex[:,0] > DurationTolerance]
    return movementepoch

def intertrial_displacement_loco_test(data,trial=None):
    if trial is None:
        trial=list(data.rawIntertrialPosition.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
    
    dis=[0]
    for i in trial:
        assert data.dataType == 'ViewPoint'
        allTraj=smooth(data.rawIntertrialPosition[i],.3*data.cameraSamplingRateInterTrial)
        posMoveIndex=_find_persistent_movement_epochs(allTraj,
                                                      TrdSpeed=0,
                                                      fs=data.cameraSamplingRateInterTrial
                                                      )
        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)))
        dis.append(0)

    return np.array(dis[:-1])

In [ ]:
def plot_intertrial_displacement_loco(root,ax,sessionList,color, x_pos=1):
    sessionSlice=slice(None,10)

    dis=np.ones((sessionSlice.stop,len(sessionList)))*np.nan
    for i,session in enumerate(sessionList):
        data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
        dis[:,i]=intertrial_displacement_loco_test(data)[sessionSlice] / 100 #meters

    height=np.nanmedian (np.nansum(dis,axis=0))
    yerr  =np.nanstd    (np.nansum(dis,axis=0))

    ax.bar(x=x_pos, height=height, width=0.75, align='center',
          edgecolor='None',color=color, alpha=.9, zorder=1)

    np.random.seed(seed=12)
    sigma=0.2
    jitter=np.random.uniform(low=x_pos-sigma, high=x_pos+sigma, size=len(sessionList))
    ax.scatter(jitter,np.nansum(dis,axis=0),s=2,c='k', marker='o', alpha=.7, zorder=2)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks([x_pos])
    ax.set_ylabel(f'{sessionSlice.stop}min Exploration (m)')
    ax.set_ylim([0,30])
    ax.set_xlim([ax.get_xlim()[0],x_pos+1])
    
    ax.tick_params(axis='x', direction='out')

In [ ]:
if "__file__" not in dir():

    LesionLocoTest=  ['Rat325','Rat337','Rat341','Rat345','Rat365','Rat366',
                      'Rat372','Rat363','Rat362','Rat355','Rat354','Rat353']

    ControlLocoTest= ['Rat359','Rat360','Rat361','Rat356','Rat357','Rat358',
                      'Rat373','Rat349', 'Rat368','Rat367','Rat369','Rat371']

    profile={'Type':'Good',
             'rewardType':'Progressive',
             'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DMS-EXP',
                    'Early-Lesion_DS-EXP','Late-Lesion_DLS-EXP',
                    'Control-EXP','ImmobileTreadmill-EXP']
             }

    lesionSessionLoco  =batch_get_session_list(root,ControlLocoTest,profile)['Sessions']
    controlSessionLoco =batch_get_session_list(root,LesionLocoTest,profile)['Sessions']

    plt.close('all')
    ax=plt.figure().add_subplot(111);

    sessionList=lesionSessionLoco
    color='r'
    plot_intertrial_displacement_loco(root,ax,sessionList,color)
    sessionList=controlSessionLoco
    color='gray'
    plot_intertrial_displacement_loco(root,ax,sessionList,color,x_pos=2)

---

Forward Speed  for Loco Test 

In [ ]:
def _forwardRunningSpeed_loco(pos, cs: int, MinDur=1, MinDisplacement=30):
    MinDuration=MinDur*cs
    posLen=len(pos)
    posDiff=np.diff(a=pos, prepend=pos[0])
    ForwardDiff=(posDiff<=0).astype(int)
    edges=ForwardDiff - np.concatenate((np.array([False]),ForwardDiff[:-1]))
    try:
        ForwardEpoch=np.where(edges!=0)[0].reshape(-1,2)
    except ValueError:
        ForwardEpoch=np.where(edges!=0)[0][:-1].reshape(-1,2)

    _ForwardSpeed=[]
    for i,j in ForwardEpoch:
        if (j-i < MinDuration                   #short epoch
            or pos[i]-pos[j] < MinDisplacement  #short displacement
        ):
            continue

        _ForwardSpeed.append(((pos[i]-pos[j])/(j-i))*cs)
#         _ForwardSpeed.append(np.amax(np.diff(pos[i:j]))*cs)

    return _ForwardSpeed

def loco_forward_speed (data, trdSpeed):
    cs=int(data.cameraSamplingRate)
    speeds=[]
    
    for trial,pos in data.position.items():
        if data.treadmillSpeed[trial] != trdSpeed:
            continue
        speeds.extend(_forwardRunningSpeed_loco(pos=pos, cs=cs))
        
    return np.nanmean(speeds)

In [ ]:
def plot_loco_forward_speed(root,ax,sessionList,color,offset=0):
    _trdSpeed=np.array([0,10,15,20,25,30,35,40])

    ForSpeed=np.ones((len(_trdSpeed),len(sessionList)))*np.nan
    for i,session in enumerate(sessionList):
        animal=session[:6]
        data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);

        for j,spd in enumerate(_trdSpeed):
            ForSpeed[j,i]=loco_forward_speed (data, spd)+spd
            
    ax.errorbar( _trdSpeed+offset, np.nanmedian(ForSpeed,axis=1), np.nanstd(ForSpeed,axis=1), color=color)
    ax.plot([_trdSpeed[0],_trdSpeed[-1]],[_trdSpeed[0],_trdSpeed[-1]],'k:',lw=1)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds(_trdSpeed[0],_trdSpeed[-1])
    ax.set_xlim([_trdSpeed[0]-1,_trdSpeed[-1]+1])
    ax.set_xticks(_trdSpeed)
    ax.set_ylabel(f'Opposite Speed (cm/s)')
    ax.set_ylim([0,70])

In [ ]:
if "__file__" not in dir():

    LesionLocoTest=  ['Rat325','Rat337','Rat341','Rat345','Rat365','Rat366',
                      'Rat372','Rat363','Rat362','Rat355','Rat354','Rat353']

    ControlLocoTest= ['Rat359','Rat360','Rat361','Rat356','Rat357','Rat358',
                      'Rat373','Rat349', 'Rat368','Rat367','Rat369','Rat371']

    profile={'Type':'Good',
             'rewardType':'Progressive',
             'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DMS-EXP',
                    'Early-Lesion_DS-EXP','Late-Lesion_DLS-EXP',
                    'Control-EXP','ImmobileTreadmill-EXP']
             }

    lesionSessionLoco  =batch_get_session_list(root,ControlLocoTest,profile)['Sessions']
    controlSessionLoco =batch_get_session_list(root,LesionLocoTest,profile)['Sessions']

    plt.close('all')
    ax=plt.figure().add_subplot(111);

    sessionList=lesionSessionLoco
    color='r'
    plot_loco_forward_speed(root,ax,sessionList,color)
    sessionList=controlSessionLoco
    color='gray'
    plot_loco_forward_speed(root,ax,sessionList,color,offset=.2)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:red'
    DMSColor='xkcd:blue'
    DSColor='xkcd:magenta'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
       

    
    #===============================================
    
    # GRID 1 PARAMS
    
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }

    profileLateLesions1={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': ['Late-Lesion_DLS',
                             'Late-Lesion_DMS','Late-Lesion_DMS-Sharp',
                             'Late-Lesion_DS','Late-Lesion_DS-Sharp']
                     }

    lesionSlice1 =slice(-5,None)
    ControlSlice1=slice(-5,None)

    
    
    #===============================================
    
    # GRID 2 PARAMS
    
    session2='Rat366_2019_10_02_10_50'
    trialsRange2=range(90,99)
    
    
    #===============================================
    
    # GRID 3 PARAMS


    
    
    #===============================================
    
    # GRID 4 PARAMs
    
    profilePre4={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse','Reverse-AfterBreak']
             }
    profilePost4={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse-Late_DLS']
             }

    TaskParamToPlot4='median entrance time (sec)'
    color4='xkcd:turquoise'
    
    Profiles4=(profilePre4,profilePost4,)
    preSlice4=slice(-3,None)
    postSlice4=slice(0,3)
    finSlice4=slice(7,10)
    

    #===============================================
    
    # GRID 5 PARAMs
    
    TaskParamToPlot5 = _anti_sequential_trials.__name__
    

    
    #===============================================
    
    # GRID 6 PARAMs

    LesionLocoTest6=  ['Rat325','Rat337','Rat341','Rat345','Rat365','Rat366',
                      'Rat372','Rat363','Rat362','Rat355','Rat354','Rat353']

    ControlLocoTest6= ['Rat359','Rat360','Rat361','Rat356','Rat357','Rat358',
                      'Rat373','Rat349', 'Rat368','Rat367','Rat369','Rat371']

    profile6={'Type':'Good',
             'rewardType':'Progressive',
             'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DMS-EXP',
                    'Early-Lesion_DS-EXP','Late-Lesion_DLS-EXP',
                    'Control-EXP','ImmobileTreadmill-EXP']
             }

    lesionSessionLoco6  =batch_get_session_list(root,LesionLocoTest,profile)['Sessions']
    controlSessionLoco6 =batch_get_session_list(root,ControlLocoTest,profile)['Sessions']
    colorLesion6='xkcd:plum'


Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(6,5)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        

    ##########################################
    # 1: Speed MaxPos controlled
    gs1= fig.add_gridspec(nrows=1, ncols=2, left=0.4, bottom=0.8, right=0.9, top=1, wspace=0)
    ax1l= fig.add_subplot(gs1[0])
    ax1r= fig.add_subplot(gs1[1])
    axes1=(ax1l,ax1r)
    
    pos1l,*_=plot_animal_maxPos_speed_relation(root, ax1l, profilePreLesion1, {},
                                               Slice=ControlSlice1,animal_plot=True)
    pos1r,*_=plot_animal_maxPos_speed_relation(root, ax1r, profileLateLesions1, ColorCode, 
                                               Slice=lesionSlice1,animal_plot=True)
    
    ax1l.text(0,1,f' n={len(pos1l)} rats',
              ha='left',va='top',fontsize='xx-small',transform=ax1l.transAxes)
    ax1r.text(0,1,f' n={len(pos1r)} rats',
              ha='left',va='top',fontsize='xx-small',transform=ax1r.transAxes)
    
    for ax in axes1:
        ax.set_ylim([10,90])
        ax.spines['left'].set_bounds(10,90)
        ax.set_yticks(range(10,91,20))
        ax.set_ylabel('')
        
    ax1l.set_ylabel('Speed (cm/s)')
    ax1r.set_yticklabels('')
    
    totAx1=fig.add_subplot(gs1[:],frameon=False)
    totAx1.set_xlabel('Max. Pos. on treadmill')
    totAx1.yaxis.set_visible(False)
    totAx1.tick_params(color=(0, 0, 0, 0),labelcolor=(0, 0, 0, 0),zorder=-10)



    ##########################################
    # 1.5: MaxPos Speed Def
    gs1_5= fig.add_gridspec(nrows=1, ncols=1, left=.1, bottom=0.8, right=.3, top=1)
    ax1_5= fig.add_subplot(gs1_5[0])
    plot_speed_pos_def(ax1_5)

    
    ##########################################
    # 2: plot consecutive trials for Reverse Treadmill
    gs2= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=.45, right=0.74, top=.7)
    ax2= fig.add_subplot(gs2[0])

    plot_consecutive_trajectories(root, session2, trialsRange2, ax2)
    
    
    

    ##########################################
    # 3: plot single session trajectories
    gs3= fig.add_gridspec(nrows=1, ncols=1, left=.75, bottom=0.45, right=1, top=.7)
    ax3= fig.add_subplot(gs3[0])

    plot_trajectories_and_distributions(root, ax3, session2)
#     add_legend_for_consecutive_trajectories(ax3)
    
    ##########################################
    # 4: reverse lesion event plot ET
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=0.1, right=0.2, top=.35)
    ax4= fig.add_subplot(gs4[0])
    
    out4=plot_event_1on1(root, ax4, Profiles4, colorCode={'':color4,'DLS':color4}, badAnimals=None, x_pos=(1,2,3),
                         TaskParamToPlot=TaskParamToPlot4, nPre=preSlice4, nPost=postSlice4,nFin=finSlice4)

    ax4.text(0,1,f' n={len(out4[1])} rats',
          ha='left',va='top',fontsize='xx-small',transform=ax4.transAxes)

    ax4.axhline(y=7, ls=':',c='gray',lw=1, zorder=-5)
    ax4.set_ylim([5,10])
    ax4.spines['left'].set_bounds(5,10)
    ax4.set_yticks(np.arange(5,11))
    ax4.set_yticklabels([5,'',7,'','',10])
    ax4.set_ylabel('Entrance time (s)')

    
    
    ##########################################
    # 5: reverse lesion event plot ANTI SEQ
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=.28, bottom=0.1, right=0.48, top=.35)
    ax5= fig.add_subplot(gs5[0])
    
    out5=plot_event_1on1(root, ax5, Profiles4, colorCode={'':color4,'DLS':color4}, badAnimals=None, x_pos=(1,2,3),
                         TaskParamToPlot=TaskParamToPlot5, nPre=preSlice4, nPost=postSlice4,nFin=finSlice4)


    ax5.set_ylim([0,1])
    ax5.spines['left'].set_bounds(0,1)
    ax5.set_yticks(np.arange(0,1.01,.2))
    ax5.yaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax5.set_ylabel('%Anti Routine')


    ##########################################
    # 6: free exploration
    gs6= fig.add_gridspec(nrows=1, ncols=1, left=0.56, bottom=0.1, right=.68, top=.35)
    ax6= fig.add_subplot(gs6[0])

    plot_intertrial_displacement_loco(root,ax6,controlSessionLoco6,color=CtrlColor,x_pos=1)
    plot_intertrial_displacement_loco(root,ax6,lesionSessionLoco6,color=colorLesion6,x_pos=2)
    
    ax6.text(1,1,f'n={len(controlSessionLoco6)}',
              ha='center',va='bottom',fontsize='xx-small')
    ax6.text(2,1,f'n={len(lesionSessionLoco6)}',
              ha='center',va='bottom',fontsize='xx-small')

    
    ax6.set_xlim([0.38,2.62])
    ax6.set_xticks([1,2])
    ax6.set_xticklabels(['Control','Lesion'])
    
    
#     ##########################################
#     # 3: % speed correlation
#     gs3= fig.add_gridspec(nrows=1, ncols=1, left=0.77, bottom=0, right=1, top=1)
#     ax3= fig.add_subplot(gs3[0])
  
#     behav3, size3, animals3=late_lesion_correlation_with_size(root, ax=ax3, Profiles=Profiles3, badAnimals=[],
#                                                               color=ColorCode,
#                                                               TaskParamToPlot=TaskParamToPlot1,
#                                                               preSlice=preSlice3, postSlice=postSlice3)
    
#     r3,p3=stats.pearsonr(size3, behav3)
#     s=f'$n={len(animals3)}$ rats'+'\n'+'$r=$'+'$%+.2f$'%(r3)+'\n'+'$p=$'+'{}'.format(SciNote(p3))
#     ax3.text(1,10,s, ha='right',va='top', fontsize='xx-small')
    
#     ax3.set_ylim([-52,12])
#     ax3.spines['left'].set_bounds(-50,10)
#     ax3.set_yticks([-50,-40,-30,-20,-10,0,10])
#     ax3.set_yticklabels([-50,'',-30,'',-10,'',10])
#     ax3.set_ylabel('$\Delta$Speed (cm/s)')
    
    


    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(axes4[0],ax1,ax2,ax5,ax7,ax8,ax6)
#     OFFX=np.array([.07]*len(AXES))
#     OFFY=np.array([.01]*len(AXES))
#     OFFX[5]=0.03
#     OFFX[[0,1,2,4,6]]=0.05
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','MotorPreserved.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()